# Analysis

In [1]:
import pandas as pd

In [2]:
patient_diagnosis = pd.DataFrame.from_csv("Patient_Diagnosis.csv")
patient_diagnosis.head(2)

,Diagnosis_Date,Diagnosis_Code,Diagnosis
Patient_ID,,,
2038,2010-01-21,174.9,Breast Cancer
2120,2010-01-09,174.1,Breast Cancer


In [3]:
patient_treatment = pd.DataFrame.from_csv("Patient_Treatment.csv")
patient_treatment.head(2)

,Treatment_Date,Drug_Code
Patient_ID,,
2038,2010-02-14,201
2038,2010-02-21,201


### 1.  How many patients does the clinic have for each cancer diagnosis?
Time: 3 minutes

In [220]:
print "Total Patients: ", len(patient_diagnosis.index.unique())

Total Patients:  37


In [221]:
print "Unique Patients per Cancer Diagnosis: "
patient_diagnosis.reset_index().groupby("Diagnosis").Patient_ID.nunique()

Unique Patients per Cancer Diagnosis: 


Diagnosis
Breast Cancer    26
Colon Cancer     12
Name: Patient_ID, dtype: int64

In [222]:
print "Unique Patients per Cancer Diagnosis Code: "
patient_diagnosis.reset_index().groupby("Diagnosis_Code").Patient_ID.nunique()

Unique Patients per Cancer Diagnosis Code: 


Diagnosis_Code
153.3     2
153.4     3
153.5     2
153.6     1
153.7     1
153.8     1
153.9     2
174.1     4
174.2     1
174.3     2
174.4     1
174.5     2
174.6     1
174.7     2
174.8     3
174.9    10
Name: Patient_ID, dtype: int64

### 2.  How long after being diagnosed does it take for a patient to start treatment?  Do any patients start treatment before being diagnosed?

This involved determining the time between diagnosis and the first treatment date for each patient and then taking an average. For the second question I looked for treatment dates that happened before diagnosis dates.

Time: 10 minutes

In [223]:
patient_composite = patient_diagnosis.join(patient_treatment)

# Change types of date columns to datetime
patient_composite['Treatment_Date'] = pd.to_datetime(patient_composite['Treatment_Date'], format='%Y-%m-%d')
patient_composite['Diagnosis_Date'] = pd.to_datetime(patient_composite['Diagnosis_Date'], format='%Y-%m-%d')

# Add Time Between Diagnosis / Treatment
patient_composite["Days_Btwn_Diagnosis_Treatment"] = (patient_composite["Treatment_Date"] - 
                                                      patient_composite["Diagnosis_Date"]).astype('timedelta64[D]')
patient_composite.head(2)

,Diagnosis_Date,Diagnosis_Code,Diagnosis,Treatment_Date,Drug_Code,Days_Btwn_Diagnosis_Treatment
Patient_ID,,,,,,
2038,2010-01-21,174.9,Breast Cancer,2010-02-14,201,24
2038,2010-01-21,174.9,Breast Cancer,2010-02-21,201,31


In [224]:
patient_composite_distinct_treatment_start = patient_composite.reset_index().groupby("Patient_ID").min()
treatment_days = patient_composite_distinct_treatment_start["Days_Btwn_Diagnosis_Treatment"].mean()
print "For a patient who receives treatment, it takes an average of {n} days to begin treatment.".format(n=round(treatment_days,1))


For a patient who receives treatment, it takes an average of 26.9 days to begin treatment.


In [225]:
treatment_before_diagnosis = patient_composite[patient_composite["Days_Btwn_Diagnosis_Treatment"] < 0]
patients_treatment_before = list(treatment_before_diagnosis.index.unique())
print "{n} Patients started treatment before diagnosis: {patients}".format(n=len(patients_treatment_before), 
                                                                       patients = patients_treatment_before) 

2 Patients started treatment before diagnosis: [4374, 6877]


### 3.  Is there a difference between Drug 201 and Drug 202 in terms of length of treatment?

I looked at the first and final treatment day, grouped by patient and drug. 
Then I took the average of the differences between the final and first treatment day.

Time: 10 minutes

In [226]:
print "Treatment with drug 201 takes approximately 61 days longer on average. \n"
print "Average length of treatment (in days): "
patient_composite_treatment_max_min = patient_composite.reset_index().groupby(["Patient_ID","Drug_Code"]).agg(['max','min'])["Days_Btwn_Diagnosis_Treatment"]
patient_composite_treatment_max_min["treatment_time"] = patient_composite_treatment_max_min["max"] - patient_composite_treatment_max_min["min"]
patient_composite_treatment_max_min.reset_index().groupby("Drug_Code").mean()["treatment_time"]

Treatment with drug 201 takes approximately 61 days longer on average. 

Average length of treatment (in days): 


Drug_Code
201    72.000000
202    10.769231
Name: treatment_time, dtype: float64